In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% validation
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    "../../data/mri",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    "../../data/mri",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 19114 images belonging to 2 classes.
Found 4777 images belonging to 2 classes.


In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base for now

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


2025-04-06 02:17:48.848367: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-06 02:17:48.848674: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-06 02:17:48.848680: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-06 02:17:48.848732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-06 02:17:48.849094: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=20, steps_per_epoch=100)


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 79s 742ms/step - accuracy: 0.9993 - loss: 0.0080 - val_accuracy: 0.9998 - val_loss: 0.0013
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 62s 628ms/step - accuracy: 1.0000 - loss: 9.5400e-04 - val_accuracy: 1.0000 - val_loss: 4.8117e-04
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 57s 579ms/step - accuracy: 1.0000 - loss: 4.6405e-04 - val_accuracy: 1.0000 - val_loss: 3.0470e-04
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 57s 576ms/step - accuracy: 1.0000 - loss: 4.8695e-04 - val_accuracy: 1.0000 - val_loss: 5.7114e-04
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 563ms/step - accuracy: 1.0000 - loss: 2.3476e-04 - val_accuracy: 1.0000 - val_loss: 3.3022e-04
Epoch 6/20
 98/100 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 1.0000 - loss: 8.9993e-05

/Users/vladt/miniconda3/envs/metal/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 511ms/step - accuracy: 1.0000 - loss: 8.9684e-05 - val_accuracy: 0.9998 - val_loss: 4.5777e-04
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 560ms/step - accuracy: 1.0000 - loss: 7.5477e-05 - val_accuracy: 1.0000 - val_loss: 1.9522e-04
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - accuracy: 1.0000 - loss: 5.2531e-05 - val_accuracy: 1.0000 - val_loss: 1.7151e-04
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 55s 557ms/step - accuracy: 1.0000 - loss: 1.1504e-04 - val_accuracy: 1.0000 - val_loss: 1.8248e-04
Epoch 10/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 55s 555ms/step - accuracy: 1.0000 - loss: 9.1603e-05 - val_accuracy: 1.0000 - val_loss: 1.5314e-04
Epoch 11/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 533ms/step - accuracy: 1.0000 - loss: 3.2023e-05 - val_accuracy: 0.9998 - val_loss: 2.5002e-04
Epoch 12/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 474ms/step - accuracy: 1.0000 - loss: 4.0025e-05 - val_accuracy: 1.0000 - val_loss: 1.4030e-04
Epoch 13/20
100/100 ━━━━━━━━━━━━━━━

In [7]:
model.save("trained-models/mac-model.h5")


In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("../../images/hand.png", target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
print("MRI" if pred[0][0] > 0.5 else "Not MRI")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Not MRI
